# model creation (using API)

https://docs.aws.amazon.com/sagemaker/latest/dg/serverless-endpoints-create.html

In [1]:
# !pip3 install sagemaker
#!pip3 install --upgrade botocore

In [2]:
from abcli import fullname
from abcli import string

import abcli.logging
import logging

logger = logging.getLogger()

logger.info(f"{fullname()}, built on {string.pretty_date()}")

abcli-7.2554.1-unknown, built on 09 December 2023, 16:34:25


---

## Setup

In [3]:
import boto3
import sagemaker
region = boto3.Session().region_name
client = boto3.client("sagemaker", region_name=region)

Found credentials in shared credentials file: ~/.aws/credentials


sagemaker.config      - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config      - Not applying SDK defaults from location: /Users/kamangir/Library/Application Support/sagemaker/config.yaml


Found credentials in shared credentials file: ~/.aws/credentials


In [4]:
# Role to give SageMaker permission to access AWS services.
# https://github.com/aws-solutions-library-samples/guidance-for-training-an-aws-deepracer-model-using-amazon-sagemaker/issues/47#issuecomment-607238430
try:
    sagemaker_role = sagemaker.get_execution_role()
except ValueError:
    logger.info("sagemaker_role: defaulting to local.")
    iam = boto3.client('iam')
    sagemaker_role = iam.get_role(RoleName='Sagemaker-Access')['Role']['Arn']

sagemaker.config      - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config      - Not applying SDK defaults from location: /Users/kamangir/Library/Application Support/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name shamim to get Role path.
sagemaker_role: defaulting to local.


In [5]:
model_name = "model-2023-12-03-11-24-39-75649"
model_url = f"s3://kamangir/bolt/{model_name}.tar.gz"

In [6]:
#Get container image (prebuilt example)
from sagemaker import image_uris
container = image_uris.retrieve("xgboost", region, "0.90-1")

Defaulting to only available Python version: py3
Defaulting to only supported image scope: cpu.


In [7]:
# Create model
response = client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = sagemaker_role,
    Containers = [{
        "Image": container,
        "Mode": "SingleModel",
        "ModelDataUrl": model_url,
    }]
)

logger.info(response)

{'ModelArn': 'arn:aws:sagemaker:ca-central-1:120429650996:model/model-2023-12-03-11-24-39-75649', 'ResponseMetadata': {'RequestId': '553fe23c-4ebc-4b45-a852-c72ed100aa10', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '553fe23c-4ebc-4b45-a852-c72ed100aa10', 'content-type': 'application/x-amz-json-1.1', 'content-length': '96', 'date': 'Sun, 10 Dec 2023 00:34:28 GMT'}, 'RetryAttempts': 0}}


---

In [8]:
# END